<a href="https://colab.research.google.com/github/saltaverde/rapid/blob/master/Dijkestra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Graph Shortest Path algorithm**

---


Dijkestra's algorithm is breadth-first based search algorithm for directed graphs

In [27]:
class QueueLinkedList:
    """Implements an efficient first-in first-out Abstract Data Type using a linked List"""

    class Node:
        def __init__(self, item):
            self.item = item
            self.next = None

    def __init__(self, capacity):
        """Creates and empty queue with a capacity"""
        self.capacity = capacity  # Capacity of your queue
        self.head = None  # expect an instance of type Node - This is the starting point of the linked list
        self.num_items = 0  # number of elements in the queue

    def is_empty(self):
        """Returns true if the queue self is empty and false otherwise"""
        return self.num_items == 0

    def is_full(self):
        """Returns true if the queue self is full and false otherwise"""
        return self.num_items == self.capacity

    def enqueue(self, item):
        """Adds item to the queue"""
        if self.is_full():
            raise IndexError('Queue is full!')
        new_item = self.Node(item)
        if self.head is None:
            self.head = new_item
        else:
            node = self.head
            while node.next is not None:
                node = node.next
            node.next = new_item
        self.num_items += 1

    def dequeue(self):
        """Returns the current front of the queue"""
        if self.is_empty():
            raise IndexError('Queue is empty!')
        node = self.head
        self.num_items -= 1
        self.head = self.head.next
        return node.item

    def peek(self):
        """Returns the value of the item at the front of the queue without removing it"""
        return self.head.item

    def size(self):
        """Returns the number of elements currently in the queue, not the capacity"""
        return self.num_items


In [28]:
#@title Graph Representation
import math

class Vertex:
    def __init__(self, key):
        self.id = key
        self.connectedTo = {}

    def addNeighbor(self, nbr, weight=0):
        self.connectedTo[nbr] = weight

    def __repr__(self):
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])

    def getConnections(self):
        return self.connectedTo.keys()

    def getId(self):
        return self.id

    def getWeight(self, nbr):
        return self.connectedTo[nbr]


class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0

    def addVertex(self, key):
        self.numVertices = self.numVertices + 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex

    def getVertex(self, n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self, n):
        return n in self.vertList

    def addEdge(self, f, t, weight=1):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], weight)

    def getVertices(self):
        return self.vertList.keys()

    def __iter__(self):
        return iter(self.vertList.values())

    def addBiEdge(self, f, t, weight=1):
        self.addEdge(f, t, weight)
        self.addEdge(t, f, weight)

    def shortest_path(self, vertex):
        """Write your shortest path code here"""
        shortest = {}
        keys = self.getVertices()
        for key in keys:
            if self.vertList[key] is vertex:
                shortest[key] = [False, 0, key]
            else:
                shortest[key] = [False, math.inf, None]
        qu = QueueLinkedList(self.numVertices*2)
        qu.enqueue(vertex)
        while not qu.is_empty():
            current = qu.dequeue()
            connections = current.getConnections()
            for k in connections:
                if not shortest[k.id][0]:
                    qu.enqueue(k)
                cost = current.connectedTo[k] + shortest[current.id][1]
                if cost < shortest[k.id][1]: # or shortest[k.id][1] < 0:
                    # shortest[k.id] = [False, cost, current.id]
                    shortest[k.id][1] = cost
                    shortest[k.id][2] = current.id
            shortest[current.id][0] = True
        return self.build_path(vertex, shortest)

    def build_path(self, vertex, shortest_list):
        result = []
        keys = shortest_list.keys()
        for key in keys:
            tup = (key, [vertex.id], shortest_list[key][1])
            previous = shortest_list[key][2]
            while previous is not None and previous is not vertex.id:
                tup[1].insert(1, previous)
                previous = shortest_list[previous][2]
            tup[1].append(key)
            result.append(tup)
        return result


Now that we have the shortest path implemented, we can use it to build the 'Kevin Bacon' game.
The first function checks the bacon-degree for a given actor

In [22]:
def bacon_degree(g, actor):
    shortest = g.shortest_path(g.getVertex('Kevin Bacon'))
    actor_v = g.getVertex(actor)
    if actor_v is not None:
        for a in shortest:
            if a[0] == actor_v.id:
                return a[2]
    return -1

The second function finds the actor(s) with the highest bacon-number.

In [23]:
def highest_bacon(g):
    shortest = g.shortest_path(g.getVertex('Kevin Bacon'))
    highest = 0
    actors = set()
    for a in shortest:
        if a[2] > highest:
            actors = set()
            highest = a[2]
            actors.add((a[0], a[2]))
        elif a[2] == highest:
            actors.add((a[0], a[2]))
    return actors

The third function checks the number of connections between any pair of actors

In [24]:
def any_actors_degree(g, actor1, actor2):
    shortest = g.shortest_path(g.getVertex(actor1))
    actor_v = g.getVertex(actor2)
    if actor_v is not None:
        for a in shortest:
            if a[0] == actor_v.id:
                return a[2]
    return -1

And now for testing. First we'll build the graph:

In [25]:
g = Graph()
g.addVertex('Kevin Bacon')
g.addVertex('Tom Hanks')
g.addVertex('Bill Paxton')
g.addVertex('Paul Herbert')
g.addVertex('Yves Aubert')
g.addVertex('Shane Zaza')
g.addVertex('Seretta Wilson')
g.addVertex('Meryl Streep')
g.addVertex('John Beluci')
g.addVertex('Kathleen Quinlan')
g.addVertex('Donald Sutherland')
g.addVertex('Lloyd Bridges')
g.addVertex('Grace Kelly')
g.addVertex('Nicole Kidman')
g.addVertex('Patrick Allen')
g.addVertex('Glenn Close')
g.addVertex('John Gielgud')
g.addVertex('Vernon Dobtcheff')
g.addVertex('Kate Winslet')

g.addBiEdge('Kevin Bacon', 'Tom Hanks')
g.addBiEdge('Kevin Bacon', 'John Beluci')
g.addBiEdge('Kevin Bacon', 'Meryl Streep')
g.addBiEdge('Kevin Bacon', 'Donald Sutherland')
g.addBiEdge('Kevin Bacon', 'Bill Paxton')
g.addBiEdge('Kevin Bacon', 'Kathleen Quinlan')
g.addBiEdge('Tom Hanks', 'Kathleen Quinlan')
g.addBiEdge('Tom Hanks', 'Bill Paxton')
g.addBiEdge('Tom Hanks', 'Paul Herbert')
g.addBiEdge('Tom Hanks', 'Yves Aubert')
g.addBiEdge('Tom Hanks', 'Shane Zaza')
g.addBiEdge('Tom Hanks', 'Seretta Wilson')
g.addBiEdge('Tom Hanks', 'Lloyd Bridges')
g.addBiEdge('Lloyd Bridges', 'Grace Kelly')
g.addBiEdge('Donald Sutherland', 'Patrick Allen')
g.addBiEdge('Donald Sutherland', 'Nicole Kidman')
g.addBiEdge('Donald Sutherland', 'Vernon Dobtcheff')
g.addBiEdge('Nicole Kidman', 'Glenn Close')
g.addBiEdge('Nicole Kidman', 'John Gielgud')
g.addBiEdge('Bill Paxton', 'Kate Winslet')
g.addBiEdge('Patrick Allen', 'John Gielgud')
g.addBiEdge('Nicole Kidman', 'John Gielgud')
g.addBiEdge('Vernon Dobtcheff', 'John Gielgud')

Then we'll try out some names:

In [31]:
bacon_degree(g, 'Kevin Bacon')

0

In [32]:
bacon_degree(g, 'Tom Hanks')

1

In [33]:
bacon_degree(g, 'Kate Winslet')

2

In [34]:
bacon_degree(g, 'Glenn Close')

3